In [19]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os, getpass

load_dotenv('../.env')



True

In [20]:
def _set_env(var: str):
    value = os.environ.get(var)
    if not value:
        value = getpass.getpass(f"{var}: ")
    return value

openai_api_key = _set_env("OPENAI_API_KEY")
tavily_api_key = _set_env("TAVILY_API_KEY")
smith_api_key = _set_env("LANGCHAIN_API_KEY")


In [21]:
gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0)
gpt35_chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)


In [10]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="Hello world", name="Zoneson")

# Message list
messages = [msg]

# Invoke the model with a list of messages 
gpt4o_chat.invoke(messages)



AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_83df987f64', 'id': 'chatcmpl-BF4DfDgdeyxmycIS66PnqiykRBFy3', 'finish_reason': 'stop', 'logprobs': None}, id='run-b8f68e03-c82f-44d9-a93b-68e2da05f237-0', usage_metadata={'input_tokens': 12, 'output_tokens': 10, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [22]:
from langchain_community.tools.tavily_search import TavilySearchResults
tavily_search = TavilySearchResults(max_results=3)
search_docs = tavily_search.invoke("What is LangGraph?")


In [23]:
from typing_extensions import TypedDict

class State(TypedDict):
    graph_state: str

In [24]:
def node_1(state):
    print("---Node 1---")
    return {"graph_state": state['graph_state'] +" I am"}

def node_2(state):
    print("---Node 2---")
    return {"graph_state": state['graph_state'] +" happy!"}

def node_3(state):
    print("---Node 3---")
    return {"graph_state": state['graph_state'] +" sad!"}

In [25]:
import random
from typing import Literal

def decide_mood(state) -> Literal["node_2", "node_3"]:
    
    # Often, we will use state to decide on the next node to visit
    user_input = state['graph_state'] 
    
    # Here, let's just do a 50 / 50 split between nodes 2, 3
    if random.random() < 0.5:

        # 50% of the time, we return Node 2
        return "node_2"
    
    # 50% of the time, we return Node 3
    return "node_3"

In [27]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END

# Build graph
builder = StateGraph(State)
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)

# Logic
builder.add_edge(START, "node_1")
builder.add_conditional_edges("node_1", decide_mood)
builder.add_edge("node_2", END)
builder.add_edge("node_3", END)

# Add
graph = builder.compile()

# View
# display(Image(graph.get_graph().draw_mermaid_png()))

In [29]:
graph.invoke({"graph_state": "Hi, this is Zoneson."})


---Node 1---
---Node 2---


{'graph_state': 'Hi, this is Zoneson. I am happy!'}

In [30]:
from pprint import pprint
from langchain_core.messages import AIMessage, HumanMessage

messages = [AIMessage(content=f"So you said you were researching ocean mammals?", name="Model")]
messages.append(HumanMessage(content=f"Yes, that's right.",name="Lance"))
messages.append(AIMessage(content=f"Great, what would you like to learn about.", name="Model"))
messages.append(HumanMessage(content=f"I want to learn about the best place to see Orcas in the US.", name="Lance"))

for m in messages:
    m.pretty_print()

================================== Ai Message ==================================
Name: Model

So you said you were researching ocean mammals?
================================ Human Message =================================
Name: Lance

Yes, that's right.
================================== Ai Message ==================================
Name: Model

Great, what would you like to learn about.
================================ Human Message =================================
Name: Lance

I want to learn about the best place to see Orcas in the US.


In [31]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [32]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")
result = llm.invoke(messages)
type(result)

langchain_core.messages.ai.AIMessage

In [33]:
result


AIMessage(content='The best place to see orcas in the United States is in the Pacific Northwest, specifically around the San Juan Islands in Washington State. This region is renowned for its vibrant marine life and is one of the most reliable places in the world to spot wild orcas, often referred to as "killer whales." Here are some details:\n\n1. **San Juan Islands, Washington**: This is considered the premier location for observing orcas. The islands are part of the Orcas Island, which is one of the three main islands you can visit. The waters around these islands are home to several pods of resident orcas, especially during the summer months.\n\n2. **Seattle, Washington**: From Seattle, you can join boat tours that take you into the Puget Sound, where you might also see orcas, along with other marine animals like humpback whales and seals.\n\n3. **Anacortes, Washington**: This is another popular departure point for whale-watching tours in the San Juan Islands. It offers easy access 

In [34]:
result.pretty_print()


================================== Ai Message ==================================

The best place to see orcas in the United States is in the Pacific Northwest, specifically around the San Juan Islands in Washington State. This region is renowned for its vibrant marine life and is one of the most reliable places in the world to spot wild orcas, often referred to as "killer whales." Here are some details:

1. **San Juan Islands, Washington**: This is considered the premier location for observing orcas. The islands are part of the Orcas Island, which is one of the three main islands you can visit. The waters around these islands are home to several pods of resident orcas, especially during the summer months.

2. **Seattle, Washington**: From Seattle, you can join boat tours that take you into the Puget Sound, where you might also see orcas, along with other marine animals like humpback whales and seals.

3. **Anacortes, Washington**: This is another popular departure point for whale-watch

In [35]:
result.response_metadata


{'token_usage': {'completion_tokens': 372,
  'prompt_tokens': 67,
  'total_tokens': 439,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'model_name': 'gpt-4o-2024-08-06',
 'system_fingerprint': 'fp_898ac29719',
 'id': 'chatcmpl-BHCvqFV2GKIAN8VBW2YzJ9CXyjGZJ',
 'finish_reason': 'stop',
 'logprobs': None}

In [52]:
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

llm_with_tools = llm.bind_tools([multiply])

In [49]:
tool_call = llm_with_tools.invoke([HumanMessage(content=f"What is 2 multiplied by 3", name="Lance")])
type(tool_call)

langchain_core.messages.ai.AIMessage

In [51]:
tool_call.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  multiply (call_P86F9k2SkfeQAPVo95dAzjGZ)
 Call ID: call_P86F9k2SkfeQAPVo95dAzjGZ
  Args:
    a: 2
    b: 3


In [39]:
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage

class MessagesState(TypedDict):
    messages: list[AnyMessage]

In [40]:
from typing import Annotated
from langgraph.graph.message import add_messages

class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [41]:
from langgraph.graph import MessagesState

class MessagesState(MessagesState):
    # Add any keys needed beyond messages, which is pre-built 
    pass

In [42]:
# Initial state
initial_messages = [AIMessage(content="Hello! How can I assist you?", name="Model"),
                    HumanMessage(content="I'm looking for information on marine biology.", name="Lance")
                   ]

# New message to add
new_message = AIMessage(content="Sure, I can help with that. What specifically are you interested in?", name="Model")

# Test
add_messages(initial_messages , new_message)

[AIMessage(content='Hello! How can I assist you?', additional_kwargs={}, response_metadata={}, name='Model', id='9e55282a-5cf1-40e0-9726-39b5f5d87c9a'),
 HumanMessage(content="I'm looking for information on marine biology.", additional_kwargs={}, response_metadata={}, name='Lance', id='f5e3e7f0-4045-42d6-980f-2d63c091f58d'),
 AIMessage(content='Sure, I can help with that. What specifically are you interested in?', additional_kwargs={}, response_metadata={}, name='Model', id='2f6d3242-a5c0-4927-95d7-b17f4356a4fe')]

In [43]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
    
# Node
def tool_calling_llm(state: MessagesState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# Build graph
builder = StateGraph(MessagesState)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_edge(START, "tool_calling_llm")
builder.add_edge("tool_calling_llm", END)
graph = builder.compile()

# View
# display(Image(graph.get_graph().draw_mermaid_png()))

In [44]:
messages = graph.invoke({"messages": HumanMessage(content="Hello!")})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Hello!
================================== Ai Message ==================================

Hello! How can I assist you today?


In [45]:
messages = graph.invoke({"messages": HumanMessage(content="Multiply 2 and 3")})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Multiply 2 and 3
================================== Ai Message ==================================
Tool Calls:
  multiply (call_5U7fI75SpXYV5H0DLOaN0y0a)
 Call ID: call_5U7fI75SpXYV5H0DLOaN0y0a
  Args:
    a: 2
    b: 3


In [53]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

# Node
def tool_calling_llm(state: MessagesState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# Build graph
builder = StateGraph(MessagesState)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode([multiply]))
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", END)
graph = builder.compile()

# View
# display(Image(graph.get_graph().draw_mermaid_png()))

In [54]:
from langchain_core.messages import HumanMessage
messages = [HumanMessage(content="Hello world.")]
messages = graph.invoke({"messages": messages})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Hello world.
================================== Ai Message ==================================

Hello! How can I assist you today?


In [55]:
messages = [HumanMessage(content=f"What is 2 multiplied by 3", name="Lance")]
messages = graph.invoke({"messages": messages})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================
Name: Lance

What is 2 multiplied by 3
================================== Ai Message ==================================
Tool Calls:
  multiply (call_gobZRgTl3KMutUcDpzLG057B)
 Call ID: call_gobZRgTl3KMutUcDpzLG057B
  Args:
    a: 2
    b: 3
================================= Tool Message =================================
Name: multiply

6


In [56]:
_set_env("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain-academy"

In [57]:
from langchain_openai import ChatOpenAI

def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

# This will be a tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

def divide(a: int, b: int) -> float:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b

tools = [add, multiply, divide]
llm = ChatOpenAI(model="gpt-4o")

# For this ipynb we set parallel tool calling to false as math generally is done sequentially, and this time we have 3 tools that can do math
# the OpenAI model specifically defaults to parallel tool calling for efficiency, see https://python.langchain.com/docs/how_to/tool_calling_parallel/
# play around with it and see how the model behaves with math equations!
llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)

In [58]:
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage

# System message
sys_msg = SystemMessage(content="You are a helpful assistant tasked with performing arithmetic on a set of inputs.")

# Node
def assistant(state: MessagesState):
   return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

In [59]:
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display

# Graph
builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "assistant")
react_graph = builder.compile()

# Show
# display(Image(react_graph.get_graph(xray=True).draw_mermaid_png()))

In [60]:
messages = [HumanMessage(content="Add 3 and 4. Multiply the output by 2. Divide the output by 5")]
messages = react_graph.invoke({"messages": messages})

In [61]:
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Add 3 and 4. Multiply the output by 2. Divide the output by 5
================================== Ai Message ==================================
Tool Calls:
  add (call_fyKmLENhMDCMtfuaRdLea8CX)
 Call ID: call_fyKmLENhMDCMtfuaRdLea8CX
  Args:
    a: 3
    b: 4
================================= Tool Message =================================
Name: add

7
================================== Ai Message ==================================
Tool Calls:
  multiply (call_BtxQmxmDH4XvnhHaFulbvcWe)
 Call ID: call_BtxQmxmDH4XvnhHaFulbvcWe
  Args:
    a: 7
    b: 2
================================= Tool Message =================================
Name: multiply

14
================================== Ai Message ==================================
Tool Calls:
  divide (call_lqllO4Cikud5drzYBvBEtGXJ)
 Call ID: call_lqllO4Cikud5drzYBvBEtGXJ
  Args:
    a: 14
    b: 5
================================= Tool Message ===============

In [62]:
messages = [HumanMessage(content="Multiply that by 2.")]
messages = react_graph.invoke({"messages": messages})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Multiply that by 2.
================================== Ai Message ==================================

Could you please specify the number you would like to multiply by 2?


In [63]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()
react_graph_memory = builder.compile(checkpointer=memory)

In [64]:
# Specify a thread
config = {"configurable": {"thread_id": "1"}}

# Specify an input
messages = [HumanMessage(content="Add 3 and 4.")]

# Run
messages = react_graph_memory.invoke({"messages": messages},config)
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Add 3 and 4.
================================== Ai Message ==================================
Tool Calls:
  add (call_JS0dfGNd5mKBUH4cMH6CvLSW)
 Call ID: call_JS0dfGNd5mKBUH4cMH6CvLSW
  Args:
    a: 3
    b: 4
================================= Tool Message =================================
Name: add

7
================================== Ai Message ==================================

The sum of 3 and 4 is 7.


In [65]:
messages = [HumanMessage(content="Multiply that by 2.")]
messages = react_graph_memory.invoke({"messages": messages}, config)
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Add 3 and 4.
================================== Ai Message ==================================
Tool Calls:
  add (call_JS0dfGNd5mKBUH4cMH6CvLSW)
 Call ID: call_JS0dfGNd5mKBUH4cMH6CvLSW
  Args:
    a: 3
    b: 4
================================= Tool Message =================================
Name: add

7
================================== Ai Message ==================================

The sum of 3 and 4 is 7.
================================ Human Message =================================

Multiply that by 2.
================================== Ai Message ==================================
Tool Calls:
  multiply (call_fQSYpiIXQmWBWfaguMGpUt8r)
 Call ID: call_fQSYpiIXQmWBWfaguMGpUt8r
  Args:
    a: 7
    b: 2
================================= Tool Message =================================
Name: multiply

14
================================== Ai Message ==================================

The result of multipl

In [1]:
import platform


In [8]:
from langgraph_sdk import get_client

In [9]:
# Replace this with the URL of your own deployed graph
URL = "http://localhost:2024"
client = get_client(url=URL)

# Search all hosted graphs
assistants = await client.assistants.search()

In [10]:
assistants


[{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
  'graph_id': 'agent',
  'config': {},
  'metadata': {'created_by': 'system'},
  'name': 'agent',
  'created_at': '2025-04-04T05:00:40.729395+00:00',
  'updated_at': '2025-04-04T05:00:40.729395+00:00',
  'version': 1},
 {'assistant_id': '228f9934-0cdd-5383-92c8-ee8422522cc2',
  'graph_id': 'router',
  'config': {},
  'metadata': {'created_by': 'system'},
  'name': 'router',
  'created_at': '2025-04-04T05:00:40.682886+00:00',
  'updated_at': '2025-04-04T05:00:40.682886+00:00',
  'version': 1},
 {'assistant_id': '28d99cab-ad6c-5342-aee5-400bd8dc9b8b',
  'graph_id': 'simple_graph',
  'config': {},
  'metadata': {'created_by': 'system'},
  'name': 'simple_graph',
  'created_at': '2025-04-04T05:00:39.847517+00:00',
  'updated_at': '2025-04-04T05:00:39.847517+00:00',
  'version': 1}]

In [12]:
agent = assistants[0]
agent

{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'graph_id': 'agent',
 'config': {},
 'metadata': {'created_by': 'system'},
 'name': 'agent',
 'created_at': '2025-04-04T05:00:40.729395+00:00',
 'updated_at': '2025-04-04T05:00:40.729395+00:00',
 'version': 1}

In [13]:
# We create a thread for tracking the state of our run
thread = await client.threads.create()

In [14]:
from langchain_core.messages import HumanMessage

# Input
input = {"messages": [HumanMessage(content="Multiply 3 by 2.")]}

# Stream
async for chunk in client.runs.stream(
        thread['thread_id'],
        "agent",
        input=input,
        stream_mode="values",
    ):
    if chunk.data and chunk.event != "metadata":
        print(chunk.data['messages'][-1])

{'content': 'Multiply 3 by 2.', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '24a41091-4c7b-4ea3-9dde-f06f9a5e4030', 'example': False}
{'content': '', 'additional_kwargs': {'tool_calls': [{'id': 'call_4fjpMyfpjNdBIyjqizE2WJMc', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, 'response_metadata': {'token_usage': {'completion_tokens': 18, 'prompt_tokens': 135, 'total_tokens': 153, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_55d88aaf2f', 'id': 'chatcmpl-BNNrcOMXO5MqJkVBDYKlnEgTWTB8m', 'finish_reason': 'tool_calls', 'logprobs': None}, 'type': 'ai', 'name': None, 'id': 'run-4e82a4b8-d659-4632-b0a5-af0c482bb11e-0', 'example': False, 'tool_calls': [{'name': 'multiply', 'args': 